In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
subject = 'Rio Grande Do Sul - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [9]:
data = pd.read_csv('2003_01_model_input_RS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t),Rio Grande Do Sul - Consumo de Cimento (t)
0,2003,4.719016,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,0.775641,8.192273,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,1576.618236,2103.000000
1,2004,4.572893,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.776178,8.121313,2.093709e+08,8.969484e+06,18.258966,1.967326e+08,1530.612911,2116.716000
2,2005,4.580017,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.776738,8.096895,2.119422e+08,9.019560e+06,18.316116,1.973689e+08,1526.253731,1966.059000
3,2006,4.941856,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,0.777325,8.011393,2.145089e+08,9.086122e+06,18.398786,1.982320e+08,1625.631583,2106.926000
4,2007,5.580982,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,0.777940,7.986478,2.168705e+08,9.162867e+06,18.479506,1.990254e+08,1786.663086,2280.943000
5,2008,5.988218,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,0.778585,7.956795,2.188016e+08,9.232116e+06,18.525103,1.994837e+08,1892.120992,2718.542000
6,2009,6.131171,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,0.779263,7.985037,2.203195e+08,9.292192e+06,18.558136,1.997999e+08,1987.964633,2685.740000
7,2010,6.216180,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,0.779976,7.949439,2.258308e+08,9.359876e+06,18.639011,2.009995e+08,2093.049582,3070.312000
8,2011,6.262114,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,0.780726,7.936698,2.307298e+08,9.370894e+06,18.687305,2.017727e+08,2163.054486,3399.177000
9,2012,6.449831,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,0.781171,6.613682,2.352214e+08,9.350813e+06,18.711648,2.024297e+08,2145.073736,3487.789000


In [10]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
0,4.719016,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,0.775641,8.192273,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,1576.618236
1,4.572893,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.776178,8.121313,2.093709e+08,8.969484e+06,18.258966,1.967326e+08,1530.612911
2,4.580017,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.776738,8.096895,2.119422e+08,9.019560e+06,18.316116,1.973689e+08,1526.253731
3,4.941856,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,0.777325,8.011393,2.145089e+08,9.086122e+06,18.398786,1.982320e+08,1625.631583
4,5.580982,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,0.777940,7.986478,2.168705e+08,9.162867e+06,18.479506,1.990254e+08,1786.663086
5,5.988218,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,0.778585,7.956795,2.188016e+08,9.232116e+06,18.525103,1.994837e+08,1892.120992
6,6.131171,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,0.779263,7.985037,2.203195e+08,9.292192e+06,18.558136,1.997999e+08,1987.964633
7,6.216180,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,0.779976,7.949439,2.258308e+08,9.359876e+06,18.639011,2.009995e+08,2093.049582
8,6.262114,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,0.780726,7.936698,2.307298e+08,9.370894e+06,18.687305,2.017727e+08,2163.054486
9,6.449831,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,0.781171,6.613682,2.352214e+08,9.350813e+06,18.711648,2.024297e+08,2145.073736


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2116.716000
1     1966.059000
2     2106.926000
3     2280.943000
4     2718.542000
5     2685.740000
6     3070.312000
7     3399.177000
8     3487.789000
9     3505.720000
10    3226.786714
11    2978.617371
12    2744.168914
13    2703.746000
14    2665.575000
15    2753.564000
16    2993.250000
17    3222.295000
18    3135.029000
19            NaN
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
0,-1.436522,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.632255,1.182346,-1.618491,-1.821147,-2.128488,-1.958357,-0.701822
1,-1.570347,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.453138,1.015589,-1.421437,-1.362802,-1.819374,-1.670491,-0.885458
2,-1.563823,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.266062,0.958207,-1.236614,-0.994708,-1.479737,-1.397152,-0.902858
3,-1.232437,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.070292,0.757276,-1.052126,-0.505440,-0.988435,-1.026379,-0.506179
4,-0.647100,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.865094,0.698725,-0.882374,0.058684,-0.508726,-0.685546,0.136598
5,-0.274137,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.649732,0.628969,-0.743570,0.567715,-0.237746,-0.488677,0.557546
6,-0.143215,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.423472,0.695339,-0.634462,1.009311,-0.041436,-0.352848,0.940117
7,-0.065361,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.185580,0.611681,-0.238315,1.506834,0.439197,0.162461,1.359577
8,-0.023293,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.064679,0.581739,0.113822,1.587817,0.726199,0.494609,1.639010
9,0.148625,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.213398,-2.527373,0.436677,1.440210,0.870870,0.776819,1.567237


In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2116.716000
1     1966.059000
2     2106.926000
3     2280.943000
4     2718.542000
5     2685.740000
6     3070.312000
7     3399.177000
8     3487.789000
9     3505.720000
10    3226.786714
11    2978.617371
12    2744.168914
13    2703.746000
14    2665.575000
15    2753.564000
16    2993.250000
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
17,2.125084,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.083121,-1.130369,0.714993,-0.725876,-0.138731,0.085128,-0.794517


In [15]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    3222.295
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [16]:
train_input.iloc[-(len(train_input)//5):]

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
14,1.029414,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.310880,-0.669543,1.190613,-0.739637,0.674936,0.853475,-1.182509
15,1.325631,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.051749,-0.856070,1.152075,-0.716561,0.589431,0.744797,-1.341464
16,1.830538,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,0.647697,-0.933365,1.004011,-0.626306,0.280742,0.476893,-1.155978


In [17]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3761383428, 6713484, 894574447, 4008817289, 1716623992, 2028373843, 1507530654, 1003165678, 2378294101, 800372737, 2667860893, 1503127616, 2287684808, 1031205281, 2268478279, 126713652, 2334058567, 3118940361, 2002075485, 2846449256, 584164879, 1118229156, 3977554863, 793684978, 1600112531, 2822583282, 3313518097, 847559729, 1934677376, 1624369456, 197559145, 252190586, 381999928, 4079353390, 987896298, 633240143, 1602133653, 905509995, 1073401949, 2754114679, 3272965803, 1500667374, 751311797, 3603970791, 4014935226, 3070381375, 2932334966, 2177639674, 2238185176, 212908939, 3977896522, 465638423, 1102548172, 2014420129, 269133929, 3645829058, 1605726709, 3018642296, 3943883768, 3475728747, 1217582429, 4005246164, 2435380233, 2713744410, 499664234, 2198933388, 3117668632, 3891521646, 2801291646, 760416329, 847488549, 62769241, 501125279, 1784084851, 2986174323, 3644104178, 3223335383, 1422913313, 397742362, 1868597239, 2508434403, 92137551, 4129314654, 2408519993, 1817506404, 3911565

val_loss: 19173.640625


Step: 87 ___________________________________________
val_loss: 25616.013671875


Step: 88 ___________________________________________
val_loss: 5405.96240234375


Step: 89 ___________________________________________
val_loss: 19185.626953125


Step: 90 ___________________________________________
val_loss: 19173.654296875


Step: 91 ___________________________________________
val_loss: 17307.365234375


Step: 92 ___________________________________________
val_loss: 23181.021484375


Step: 93 ___________________________________________
val_loss: 19173.876953125


Step: 94 ___________________________________________
val_loss: 19173.611328125


Step: 95 ___________________________________________
val_loss: 240.20729064941406
winner_seed: 3706380020


Step: 96 ___________________________________________
val_loss: 19251.666015625


Step: 97 ___________________________________________
val_loss: 18140.341796875


Step: 98 ___________________________________________
val

In [21]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 462ms/step - loss: 7987901.0000 - val_loss: 7622001.5000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 7753372.5000 - val_loss: 21286694.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 12414888.0000 - val_loss: 7646742.5000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 7412081.5000 - val_loss: 10879753.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 7007964.5000 - val_loss: 10309501.0000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 7160757.0000 - val_loss: 11204925.0000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 6232271.0000 - val_loss: 11051025.0000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 6401415.0000 - val_loss: 12309561.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 6653196.5000 - val_los

1/1 [==============================] - 0s 14ms/step - loss: 1986125.6250 - val_loss: 2867719.0000
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1916279.3750 - val_loss: 1954173.5000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1815515.8750 - val_loss: 2590676.2500
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1828362.3750 - val_loss: 2425102.7500
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 1808886.2500 - val_loss: 2343509.7500
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 1799281.0000 - val_loss: 2323641.7500
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1736792.7500 - val_loss: 2216965.7500
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 1752669.5000 - val_loss: 2123646.2500
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 1595819.7500 - val_loss: 1773429.625

1/1 [==============================] - 0s 14ms/step - loss: 686822.1250 - val_loss: 656618.1875
Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 602936.3125 - val_loss: 833250.5625
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 528941.7500 - val_loss: 955903.0625
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 574169.3125 - val_loss: 983600.4375
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 587041.0000 - val_loss: 861501.2500
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 745202.4375 - val_loss: 811125.4375
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 548600.7500 - val_loss: 841120.6875
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 556232.5000 - val_loss: 1031662.5625
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 604763.8750 - val_loss: 907909.6875
Epoch 1

1/1 [==============================] - 0s 14ms/step - loss: 275855.1250 - val_loss: 126120.3359
Epoch 219/10000
1/1 [==============================] - 0s 13ms/step - loss: 195150.2188 - val_loss: 126234.6641
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 157734.4688 - val_loss: 157321.7969
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 221599.5625 - val_loss: 136516.8906
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 251725.3281 - val_loss: 96119.1328
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 204359.6406 - val_loss: 126517.8516
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 208820.0156 - val_loss: 118818.6250
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 315726.0625 - val_loss: 141010.9062
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 201056.0000 - val_loss: 177226.9375
Epoch 227

1/1 [==============================] - 0s 14ms/step - loss: 112043.2578 - val_loss: 121583.8047
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 151172.0156 - val_loss: 68601.4141
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 86299.1875 - val_loss: 77526.0234
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 110567.6250 - val_loss: 109962.5078
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 104670.5312 - val_loss: 81060.2109
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 124232.7969 - val_loss: 70200.7734
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 88358.1484 - val_loss: 66037.9297
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 110845.3672 - val_loss: 46333.9180
Epoch 299/10000
1/1 [==============================] - 0s 15ms/step - loss: 94346.4375 - val_loss: 76815.3750
Epoch 300/10000
1/

...

1/1 [==============================] - 0s 14ms/step - loss: 121539.7266 - val_loss: 12842.2588
Epoch 1180/10000
1/1 [==============================] - 0s 14ms/step - loss: 133025.2969 - val_loss: 15355.3447
Epoch 1181/10000
1/1 [==============================] - 0s 14ms/step - loss: 94113.3516 - val_loss: 13043.4375
Epoch 1182/10000
1/1 [==============================] - 0s 14ms/step - loss: 119839.0312 - val_loss: 11586.8330
Epoch 1183/10000
1/1 [==============================] - 0s 14ms/step - loss: 110618.5469 - val_loss: 12362.0947
Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 139600.4375 - val_loss: 11443.1758
Epoch 1185/10000
1/1 [==============================] - 0s 14ms/step - loss: 122494.0312 - val_loss: 11615.6064
Epoch 1186/10000
1/1 [==============================] - 0s 14ms/step - loss: 109394.8047 - val_loss: 16809.2969
Epoch 1187/10000
1/1 [==============================] - 0s 14ms/step - loss: 114548.5156 - val_loss: 11426.3018
Epoch 1188

Epoch 1253/10000
1/1 [==============================] - 0s 14ms/step - loss: 123938.1484 - val_loss: 8166.6772
Epoch 1254/10000
1/1 [==============================] - 0s 14ms/step - loss: 89073.3672 - val_loss: 9762.6670
Epoch 1255/10000
1/1 [==============================] - 0s 14ms/step - loss: 118324.0625 - val_loss: 6112.7461
Epoch 1256/10000
1/1 [==============================] - 0s 14ms/step - loss: 84591.7578 - val_loss: 8456.3662
Epoch 1257/10000
1/1 [==============================] - 0s 14ms/step - loss: 100023.0312 - val_loss: 5321.2769
Epoch 1258/10000
1/1 [==============================] - 0s 14ms/step - loss: 111497.1797 - val_loss: 5186.1069
Epoch 1259/10000
1/1 [==============================] - 0s 14ms/step - loss: 79835.5859 - val_loss: 8132.1099
Epoch 1260/10000
1/1 [==============================] - 0s 14ms/step - loss: 135253.9062 - val_loss: 11141.6719
Epoch 1261/10000
1/1 [==============================] - 0s 14ms/step - loss: 110408.3828 - val_loss: 8634.1553
Epo

Epoch 1327/10000
1/1 [==============================] - 0s 14ms/step - loss: 90840.0078 - val_loss: 25662.5156
Epoch 1328/10000
1/1 [==============================] - 0s 14ms/step - loss: 89306.2812 - val_loss: 26423.7344
Epoch 1329/10000
1/1 [==============================] - 0s 13ms/step - loss: 117450.6250 - val_loss: 26525.7344
Epoch 1330/10000
1/1 [==============================] - 0s 14ms/step - loss: 89775.5078 - val_loss: 27284.3848
Epoch 1331/10000
1/1 [==============================] - 0s 14ms/step - loss: 112788.8047 - val_loss: 25404.3496
Epoch 1332/10000
1/1 [==============================] - 0s 14ms/step - loss: 103487.5156 - val_loss: 24703.4766
Epoch 1333/10000
1/1 [==============================] - 0s 14ms/step - loss: 111966.8828 - val_loss: 26147.3418
Epoch 1334/10000
1/1 [==============================] - 0s 13ms/step - loss: 90941.6875 - val_loss: 27632.9434
Epoch 1335/10000
1/1 [==============================] - 0s 13ms/step - loss: 116164.0547 - val_loss: 23484.0

1/1 [==============================] - 0s 13ms/step - loss: 161614.2188 - val_loss: 89923.2500
Epoch 1401/10000
1/1 [==============================] - 0s 14ms/step - loss: 121050.9766 - val_loss: 80087.8828
Epoch 1402/10000
1/1 [==============================] - 0s 14ms/step - loss: 167481.1875 - val_loss: 84407.8750
Epoch 1403/10000
1/1 [==============================] - 0s 14ms/step - loss: 152378.2500 - val_loss: 86180.1406
Epoch 1404/10000
1/1 [==============================] - 0s 14ms/step - loss: 136822.5000 - val_loss: 89390.4141
Epoch 1405/10000
1/1 [==============================] - 0s 13ms/step - loss: 168037.4375 - val_loss: 78220.2578
Epoch 1406/10000
1/1 [==============================] - 0s 14ms/step - loss: 128072.1484 - val_loss: 82048.5938
Epoch 1407/10000
1/1 [==============================] - 0s 13ms/step - loss: 145527.3125 - val_loss: 87732.4922
Epoch 1408/10000
1/1 [==============================] - 0s 14ms/step - loss: 137942.5156 - val_loss: 81075.3828
Epoch 140

1/1 [==============================] - 0s 14ms/step - loss: 148945.0312 - val_loss: 74692.5703
Epoch 1474/10000
1/1 [==============================] - 0s 13ms/step - loss: 148209.2188 - val_loss: 74818.7188
Epoch 1475/10000
1/1 [==============================] - 0s 14ms/step - loss: 154956.4219 - val_loss: 69061.5547
Epoch 1476/10000
1/1 [==============================] - 0s 13ms/step - loss: 154038.2812 - val_loss: 73860.5547
Epoch 1477/10000
1/1 [==============================] - 0s 14ms/step - loss: 163833.6094 - val_loss: 64313.0508
Epoch 1478/10000
1/1 [==============================] - 0s 13ms/step - loss: 129519.6016 - val_loss: 70225.0156
Epoch 1479/10000
1/1 [==============================] - 0s 13ms/step - loss: 134879.8750 - val_loss: 71408.2969
Epoch 1480/10000
1/1 [==============================] - 0s 14ms/step - loss: 134979.0000 - val_loss: 64783.1055
Epoch 1481/10000
1/1 [==============================] - 0s 14ms/step - loss: 135805.8594 - val_loss: 62342.9492
Epoch 148

In [22]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[3222.295](test_target) - [[2972.597]](prediction) = 249.69807617187507


In [23]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [24]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
0,-0.929955,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.456068,1.870395,-1.525433,-1.476097,-1.507083,-1.534207,-0.853084
1,-1.148326,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.077568,1.013119,-1.055807,-1.060634,-1.143951,-1.105937,-1.076397
2,-1.137680,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.682249,0.718127,-0.615330,-0.726979,-0.744962,-0.699280,-1.097557
3,-0.596933,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.268558,-0.314828,-0.175651,-0.283486,-0.167804,-0.147667,-0.615170
4,0.358203,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.165057,-0.615828,0.228909,0.227860,0.395736,0.359404,0.166488
5,0.966793,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.620148,-0.974436,0.559713,0.689266,0.714070,0.652295,0.678389
6,1.180428,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.098269,-0.633239,0.819741,1.089547,0.944685,0.854373,1.143620
7,1.307470,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.600970,-1.063310,1.763856,1.540523,1.509310,1.621018,1.653710


train_target:


,Rio Grande Do Sul - Consumo de Cimento (t)
0,2116.716
1,1966.059
2,2106.926
3,2280.943
4,2718.542
5,2685.740
6,3070.312
7,3399.177


test_input:


...

,Rio Grande do Sul - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Sul - IDH,Rio Grande do Sul - Desemprego,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande Do Sul - Produção de Cimento (t)
17,2.125084,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.083121,-1.130369,0.714993,-0.725876,-0.138731,0.085128,-0.794517


test_target:


,Rio Grande Do Sul - Consumo de Cimento (t)
17,3222.295


1/1 [==============================] - 0s 36ms/step
Error: 249.69807617187507




[3031.443115234375,
 3413.0029296875,
 3450.638916015625,
 3348.772705078125,
 3254.363525390625,
 3325.68505859375,
 2985.23095703125,
 2705.29931640625,
 2715.2197265625,
 2972.596923828125]

In [25]:
display(targets)
display(predictions)

[3487.789,
 3505.72,
 3226.7867142857144,
 2978.6173714285715,
 2744.1689142857144,
 2703.746,
 2665.575,
 2753.564,
 2993.25,
 3222.295]

[3031.443115234375,
 3413.0029296875,
 3450.638916015625,
 3348.772705078125,
 3254.363525390625,
 3325.68505859375,
 2985.23095703125,
 2705.29931640625,
 2715.2197265625,
 2972.596923828125]

In [26]:
mae = mean_absolute_error(predictions, targets)
mae

317.0853150390625

In [27]:
porcentage = mae/np.mean(targets)
porcentage

0.10471251073561402